# Model

In [39]:
m = {
    ## Variables
    'M': 3, # valores observaveis
    'N': 2, # estados ocultos 
    
    ## Probabilities
    'A': [ # nxn
        [.5, .5],
        [.5, .5]
    ],
    'B': [ # nxm
        [.4, .3, .2],
        [.2, .3, .4]
    ],
    'P': [.5, .5] # 1xn
}

# Gerador de Trajetória

In [40]:
import random

def generate(T, model):
    draw = lambda size, p: random.choices(range(size), p)[0]
    
    q = draw(model['N'], model['P'])
    for t in range(1, T+1):
        yield draw(model['M'], model['B'][q]), q
        q = draw(model['N'], model['A'][q])

O = [n for n, _ in generate(15, m)]
O

[0, 0, 2, 1, 2, 2, 2, 2, 1, 0, 0, 1, 1, 0, 0]

# Forward-Backward

In [41]:
def alpha(t, i, O, model):
    if t == 0:
        return model['P'][i] * model['B'][i][O[t]]
    else:
        return sum([alpha(t-1, j, O, model)* model['A'][j][i] for j in range(model['N'])])* model['B'][i][O[t]]
    
def beta(t, i, O, model):
    if t == (len(O)-1):
        return 1
    else:
        return sum([model['A'][i][j] * model['B'][j][O[t+1]]*beta(t+1, j, O, model) for j in range(model['N'])])
    
print(alpha(2, 0, O, m))
print(beta(2, 0, O, m))

print('p: ', sum([alpha(len(O)-1, i, O, m) for i in range(m['N'])]))

0.009000000000000003
5.314410000000003e-07
p:  1.4348907000000008e-08


# Expectation Maximization (EM)

In [52]:
def xi(t, a, b, O, model):
    return (alpha(t, a, O, model) * model['A'][a][b] * model['B'][b][O[t+1]] * beta(t+1, b, O, model)) / sum([alpha(t, i, O, model) * model['A'][i][j] * model['B'][j][O[t+1]] * beta(t+1, j, O, model) for i in range(model['N']) for j in range(model['N'])])

def gamma(t, i, O, model):
    return sum([xi(t, i, j, O, model) for j in range(model['N'])]) 


def estimate_pi(i, O, model):
    return gamma(0, i, O, model)

def estimate_A(i,j, O, model):
    return sum([xi(t, i,j, O, model) for t in range(len(O)-2)]) / sum([gamma(t, i, O, model) for t in range(len(O)-2)])


def estimate_B(j,k, O, model):
    return sum([gamma(t, j, O, model) for t in range(len(O)-1) if O[t]==k]) / sum([gamma(t, j, O, model) for t in range(len(O)-1)])

print(estimate_pi(0, O, m))
print(estimate_A(0,0, O, m))
print(estimate_B(0,1, O, m))

0.6666666666666666
0.5043859649122807
0.2857142857142857


# Iterate

## Random Init

In [53]:
import numpy as np


def init_P(N):
    d = np.random.random(N)
    return (d / sum(d)).tolist()

def init_P2(N,M):
    return [init_P(M) for _ in range(N)]

n_value =2
m_value =3
m_new = {
    ## Variables
    'M': m_value,
    'N': n_value,
    ## Probabilities
    'A': init_P2(n_value,n_value),
    'B': init_P2(n_value,m_value),
    'P': init_P(n_value) # 1xn
}

m_new

{'M': 3,
 'N': 2,
 'A': [[0.3281315953088063, 0.6718684046911937],
  [0.6324612337013811, 0.36753876629861887]],
 'B': [[0.4480545254589664, 0.32867427045369174, 0.22327120408734186],
  [0.012706451752925771, 0.9482333701604805, 0.03906017808659372]],
 'P': [0.5228571390863379, 0.4771428609136621]}

In [54]:
print('p: ', sum([alpha(len(O)-1, i, O, m) for i in range(m['N'])]))
print('p: ', sum([alpha(len(O)-1, i, O, m_new) for i in range(m_new['N'])]))

p:  1.4348907000000008e-08
p:  2.579210540545468e-10


In [55]:
def get_estimate_for_A(O, model):
    return [
        [ round(estimate_A(i,j, O, model), 2) for j in range(model['N']) ] 
        for i in range(model['N']) 
    ]

def get_estimate_for_B(O, model):
    return [
        [ round(estimate_B(j,k, O, model), 2) for k in range(model['M']) ] 
        for j in range(model['N']) 
    ]

def get_estimate_for_Pi(O, model):
    return [ round(estimate_pi(i, O, model), 2) for i in range(model['N']) ] 

# m_new['A'] = get_estimate_for_A(O, m_new)
# m_new['B'] = get_estimate_for_B(O, m_new)
# m_new['P'] = get_estimate_for_Pi(O, m_new)
# m_new

In [56]:
n_iter = 20
for _ in range(n_iter):
    m_new['A'] = get_estimate_for_A(O, m_new)
    m_new['B'] = get_estimate_for_B(O, m_new)
    m_new['P'] = get_estimate_for_Pi(O, m_new)
    print('p: ', sum([alpha(len(O)-1, i, O, m) for i in range(m['N'])]))
    print('p: ', sum([alpha(len(O)-1, i, O, m_new) for i in range(m_new['N'])]))
    print('')

p:  1.4348907000000008e-08
p:  7.577860398776726e-08

p:  1.4348907000000008e-08
p:  9.554222108732408e-08

p:  1.4348907000000008e-08
p:  1.1079806657938958e-07

p:  1.4348907000000008e-08
p:  1.1009213064951889e-07

p:  1.4348907000000008e-08
p:  1.179044594170593e-07

p:  1.4348907000000008e-08
p:  1.1426980728961011e-07

p:  1.4348907000000008e-08
p:  1.0674442764486726e-07

p:  1.4348907000000008e-08
p:  1.2058268224677273e-07

p:  1.4348907000000008e-08
p:  1.2879977511700206e-07

p:  1.4348907000000008e-08
p:  1.2395339573187848e-07

p:  1.4348907000000008e-08
p:  1.3344518402426463e-07

p:  1.4348907000000008e-08
p:  1.3606811031747703e-07

p:  1.4348907000000008e-08
p:  1.4351372713706765e-07

p:  1.4348907000000008e-08
p:  1.3558494896305224e-07

p:  1.4348907000000008e-08
p:  1.5178542167201672e-07

p:  1.4348907000000008e-08
p:  1.4464567902311056e-07

p:  1.4348907000000008e-08
p:  1.6633612338443028e-07

p:  1.4348907000000008e-08
p:  1.6849587619711068e-07

p:  1.4348907

In [57]:
m

{'M': 3,
 'N': 2,
 'A': [[0.5, 0.5], [0.5, 0.5]],
 'B': [[0.4, 0.3, 0.2], [0.2, 0.3, 0.4]],
 'P': [0.5, 0.5]}

In [58]:
m_new

{'M': 3,
 'N': 2,
 'A': [[0.48, 0.52], [0.36, 0.64]],
 'B': [[0.79, 0.03, 0.19], [0.03, 0.49, 0.49]],
 'P': [1.0, 0.0]}